In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/buses/pune-bus-tickets"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def close_pop_up(driver):
    try:
        # Assume pop-up has a close button with a certain class or id
        pop_up_close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='close']"))
        )
        pop_up_close_button.click()
        time.sleep(1)  # Give it some time to close
    except Exception as e:
        # If no pop-up, or pop-up couldn't be closed, just pass
        pass


# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        close_pop_up(driver)
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 4):  # There are 3 pages
        try:
            load_page(driver, URL)
            
            if page >= 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('pune_bus_routes.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


Error occurred while accessing page 3: Message: 
Stacktrace:
0   chromedriver                        0x0000000103210500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x0000000103208890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000102e18538 cxxbridge1$string$len + 89424
3   chromedriver                        0x0000000102e5c878 cxxbridge1$string$len + 368784
4   chromedriver                        0x0000000102e96b7c cxxbridge1$string$len + 607124
5   chromedriver                        0x0000000102e51374 cxxbridge1$string$len + 322444
6   chromedriver                        0x0000000102e51fc4 cxxbridge1$string$len + 325596
7   chromedriver                        0x00000001031d7d2c cxxbridge1$str$ptr + 1685732
8   chromedriver                        0x00000001031dc530 cxxbridge1$str$ptr + 1704168
9   chromedriver                        0x00000001031bce08 cxxbridge1$str$ptr + 1575360
10  chromedriver                        0x00000001

In [5]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Mumbai to Pune,https://www.redbus.in/bus-tickets/mumbai-to-pune,Neeta tours and travels,Bharat Benz A/C Seater (2+2),10:15,03h 40m,13:55,3.8,INR 399,9 Seats available
1,Mumbai to Pune,https://www.redbus.in/bus-tickets/mumbai-to-pune,Purple Bus - Metrolink,Volvo AC Seater (2+2),10:15,03h 20m,13:35,3.7,359,18 Seats available
2,Mumbai to Pune,https://www.redbus.in/bus-tickets/mumbai-to-pune,Neeta tours and travels,Bharat Benz A/C Seater (2+2),10:15,04h 10m,14:25,4.0,INR 399,15 Seats available
3,Mumbai to Pune,https://www.redbus.in/bus-tickets/mumbai-to-pune,Neeta tours and travels,Bharat Benz A/C Seater (2+2),10:20,05h 50m,16:10,3.1,INR 399,28 Seats available
4,Mumbai to Pune,https://www.redbus.in/bus-tickets/mumbai-to-pune,Neeta tours and travels,Bharat Benz A/C Seater (2+2),10:20,04h 50m,15:10,3.3,INR 399,17 Seats available
...,...,...,...,...,...,...,...,...,...,...
495,Pune to Amravati,https://www.redbus.in/bus-tickets/pune-to-amra...,Online go,Non AC Seater (2+2),19:01,11h 52m,06:53,1.3,INR 2050,2 Seats available
496,Pune to Amravati,https://www.redbus.in/bus-tickets/pune-to-amra...,Naksh Travels,NON A/C Sleeper (2+1),17:00,11h 34m,04:34,1.8,INR 2150,50 Seats available
497,Pune to Amravati,https://www.redbus.in/bus-tickets/pune-to-amra...,Naksh Travels,Non AC Seater (2+2),20:00,10h 21m,06:21,4.2,INR 1900,36 Seats available
498,Pune to Amravati,https://www.redbus.in/bus-tickets/pune-to-amra...,VRL Travels,Volvo 9600 Multi-Axle A/C Sleeper (2+1),21:00,10h 00m,07:00,4.0,INR 1300,20 Seats available
